# Qiskit Experiment Classes

It includes implementions of full experiments for:
* `QSTExperiment` an example implementation of a basic experiment
* `ParallelExperiment` class for combining experiments on different qubits into a single experiment
* `BatchExperiment` class for combining arbitrary experiments into a single batch experiment

The parallel and batch experiments will be used to combine various QST experiments on different numbers of qubits

In [1]:
from qiskit import QuantumCircuit, execute
from qiskit.circuit.library import HGate, XGate
from qiskit.providers.aer import QasmSimulator


import qiskit_experiments as qe
import qiskit_experiments.tomography as tomo

backend = QasmSimulator()

## Running single experiments

First run several individual QST experiments

In [2]:
# Run QST for applying H gate to qubit-4
exp1 = tomo.QSTExperiment(HGate(), [4])
data1 = exp1.run(backend)

# View result data
data1

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 70f207c4-6e8f-4d51-b280-d3aaf7f4bf7f
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.49707036+0.00000000e+00j 0.49999142+3.90783589e-17j]
               [0.49999142-3.90783589e-17j 0.50292964+0.00000000e+00j]])

In [3]:
# Run QST for applying X gate to qubit-4
exp2 = tomo.QSTExperiment(XGate(), [1])
data2 = exp2.run(backend)

# View result data
data2

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 3c8907c6-4db0-4ca5-9dd1-b360e071ee7c
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[ 1.90733772e-06+0.j         -9.76558775e-04-0.00097656j]
               [-9.76558775e-04+0.00097656j  9.99998093e-01+0.j        ]])

In [4]:
# 2-qubit state preparation circuit
qc3 = QuantumCircuit(2)
qc3.h(0)
qc3.cx(0, 1)

# Run QST for applying circuit to qubits [3, 0]
exp3 = tomo.QSTExperiment(qc3, [3, 0])
data3 = exp3.run(backend)

# View result data
data3

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: bfdc3b2e-acd6-48cc-a8c2-cbf48f605cf1
Status: COMPLETE
Circuits: 9
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[ 0.4922344 +0.j         -0.0023835 -0.00641539j
                 0.0029571 +0.00251222j  0.49192597+0.01783698j]
               [-0.0023835 +0.00641539j  0.00392868+0.j
                 0.00104455+0.00374231j -0.00664926+0.00317927j]
               [ 0.0029571 -0.00251222j  0.00104455-0.00374231j
                 0.00394338+0.j         -0.00115189+0.00061176j]
               [ 0.49192597-0.01783698j -0.00664926-0.00317927j
                -0.00115189-0.00061176j  0.49909203+0.j        ]])

## Batch Experiments

Next we demonstrate batch experiments where several individual experiments are grouped into a single meta-experiment. Currently this just appends the circuits from each sub experiment sequentially, but in the future it should allow shuffling or interleaving circuits from individual experiments in the batch circuit list to be executed.

For a simple first demonstration we consider running multiple experiments on the same qubit (in this case the same experiment twice)

In [5]:
# Batch of two experiments both on qubit-4
batch_exp1 = qe.composite.BatchExperiment(2 * [exp1])
batch_data1 = batch_exp1.run(backend)

# View result
batch_data1

---------------------------------------------------
Experiment: BatchExperiment
Experiment ID: 89980a9b-a3a0-4055-b169-1ae3cc272440
Status: COMPLETE
Component Experiments: 2
Circuits: 6
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'QSTExperiment']
- experiment_ids: ['6084f3d8-dbe1-40d2-9345-faa6a7963961', '38fcfcc9-c201-45ed-bec7-38af92f283b9']
- experiment_qubits: [(4,), (4,)]

#### Viewing sub experiment data

The experiment data returned from a batched experiment also contains individual experiment data for each sub experiment which can be accessed using `experiment_data(index)`

In [6]:
# Print sub-experiment data
for i in range(2):
    print(batch_data1.component_experiment_data(i), '\n')

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 6084f3d8-dbe1-40d2-9345-faa6a7963961
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.49414113+0.j         0.49995709+0.00292944j]
               [0.49995709-0.00292944j 0.50585887+0.j        ]]) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 38fcfcc9-c201-45ed-bec7-38af92f283b9
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.51950801+0.j         0.49940502+0.01463101j]
               [0.49940502-0.01463101j 0.48049199+0.j        ]]) 



### Batches on differing qubits

Batch experiments don't need to be on the same qubits, each individual one can be run on arbitrary qubits.

In [7]:
batch_exp2 = qe.composite.BatchExperiment([exp1, exp2, exp1, exp3])
batch_data2 = batch_exp2.run(backend)

# Batch data
batch_data2

---------------------------------------------------
Experiment: BatchExperiment
Experiment ID: 02df03ac-5888-4f97-991f-5abe7bf2cda4
Status: COMPLETE
Component Experiments: 4
Circuits: 18
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'QSTExperiment', 'QSTExperiment', 'QSTExperiment']
- experiment_ids: ['8761af27-5e68-453c-8f4c-414b6a1ba39e', 'd9a39d8f-87bf-4c38-b864-321cd3752716', '54148db2-c764-4255-8468-254636868b68', 'a22a22dd-5864-4a3b-b8e4-b16f89f36c9e']
- experiment_qubits: [(4,), (1,), (4,), (3, 0)]

#### View sub experiment data

In [8]:
for i in range(4):
    print(batch_data2.component_experiment_data(i), '\n')

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 8761af27-5e68-453c-8f4c-414b6a1ba39e
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.5       +0.j        0.49930622-0.0263306j]
               [0.49930622+0.0263306j 0.5       +0.j       ]]) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: d9a39d8f-87bf-4c38-b864-321cd3752716
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[ 0.00153121+0.j         -0.03699573-0.01265643j]
               [-0.03699573+0.01265643j  0.99846879+0.j        ]]) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 54148db2-c764-4255-8468-254636868b68
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------

## Parallel Experiments

Another kind of meta-experiment is a *parallel experiment*. This involves combing sub experiments on different qubits into a single experiment where each sub circuit is applied to different qubits in parallel. The total number of circuits in the parallel experiment will be equal to the largest number of circuits of the individual sub experiments.

Processing is done by marginalizing the joint measurements into the exected count dictionaries defined by the classical bit registers in the individaul experiments.

### Parallel tomography example
For our example we run parallel 1 qubit QST on qubits [4] and [1], and 2-qubit QST on qubits [3, 0]. This results in 9 circuits being executed -- the number required for the 2-qubit QST experiment

In [9]:
# Define a parallel experiment
par_exp = qe.composite.ParallelExperiment([exp1, exp2, exp3])
par_data = par_exp.run(backend)

# View data
par_data

---------------------------------------------------
Experiment: ParallelExperiment
Experiment ID: f9b01823-896c-4a16-b4ef-c5c3b8a993fe
Status: COMPLETE
Component Experiments: 3
Circuits: 9
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'QSTExperiment', 'QSTExperiment']
- experiment_ids: ['8405d5af-a720-4609-bd14-e8f16ee32a23', '9e492174-58b3-4e7d-a8e5-34248f71a37d', 'f2cb74a0-c09b-4085-82a5-ffeae3f7b8f6']
- experiment_qubits: [(4,), (1,), (3, 0)]

#### View sub experiment data

In [10]:
for i in range(4):
    print(batch_data2.component_experiment_data(i), '\n')

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 8761af27-5e68-453c-8f4c-414b6a1ba39e
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.5       +0.j        0.49930622-0.0263306j]
               [0.49930622+0.0263306j 0.5       +0.j       ]]) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: d9a39d8f-87bf-4c38-b864-321cd3752716
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[ 0.00153121+0.j         -0.03699573-0.01265643j]
               [-0.03699573+0.01265643j  0.99846879+0.j        ]]) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 54148db2-c764-4255-8468-254636868b68
Status: COMPLETE
Circuits: 3
Analysis Results: 1
---------------------------------------

## Complicated Meta Experiments

We can arbitrarily combine batch experiments and parallel experiments into nested metaexperiments (so long as the parallel experiment components are defined on different qubits of course).

For example we can consider the following contribed example involving multiple layers of batch and parallel tomography experiments:

In [11]:
big_exp = qe.composite.BatchExperiment([
            qe.composite.ParallelExperiment([exp3, qe.composite.ParallelExperiment([exp1, exp2])]),
            qe.composite.BatchExperiment([exp1, exp2]),
            exp3])
big_data = big_exp.run(backend)
big_data

---------------------------------------------------
Experiment: BatchExperiment
Experiment ID: 774e31f4-cbe0-4166-968b-43b4fbfe8eda
Status: COMPLETE
Component Experiments: 3
Circuits: 24
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['ParallelExperiment', 'BatchExperiment', 'QSTExperiment']
- experiment_ids: ['5c59295f-8ec8-498d-b9ca-6291b3145304', '1e187167-a079-4600-9a4f-183942abea2d', 'f0e04804-ea1a-4ef0-8d3a-cba54acfbbab']
- experiment_qubits: [(3, 0, 4, 1), (4, 1), (3, 0)]

In [12]:
# print sub experiments
# We could also print the sub^2-experiments of the parallel and batch sub-experiments

for i in range(3):
    print(big_data.component_experiment_data(i), '\n')

---------------------------------------------------
Experiment: ParallelExperiment
Experiment ID: 5c59295f-8ec8-498d-b9ca-6291b3145304
Status: COMPLETE
Component Experiments: 2
Circuits: 9
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'ParallelExperiment']
- experiment_ids: ['b2451047-dd2d-4c17-bfc5-5e70094e55a3', '11b3194b-ecb8-4f30-9865-ea45d56b5561']
- experiment_qubits: [(3, 0), (4, 1)] 

---------------------------------------------------
Experiment: BatchExperiment
Experiment ID: 1e187167-a079-4600-9a4f-183942abea2d
Status: COMPLETE
Component Experiments: 2
Circuits: 6
Analysis Results: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'QSTExperiment']
- experiment_ids: ['7712035e-187b-4ff4-bf5c-f8804c8c2f36', 'a532322f-cf5c-49b8-894f-584cb4c118e9']
- experiment_qubits: [(4,), (1,)] 

--------------------------------------------------